In [1]:
import numpy as np
import cv2

## 히스토그램 구하기 

영상의 히스토그램(histogram) : 영상의 픽셀 값 분포를  
그래프 형태로 표현한 것을 의미 

GrayScale 영상의 경우, 각 GrayScale 값에 해당하는  
픽셀의 개수를 구하고 이를 막대 그래프 형태로 표현함으로써  
히스토그램을 구할 수 있음 

컬러 영상에 대해서도 세 개의 색상 성분 조합에 따른  
픽셀 개수를 계산하여 히스토그램을 구할 수 있음 

왼쪽에 나타난 4x4 입력 영상은, 각 픽셀이 0부터 7 사이에 밝기를 가질 수 있는  
단순한 형태의 영상 

각각의 밝기에 해당하는 픽셀 개수를 세어서  
막대그래프 형태로 표현한 히스토그램을 오른쪽에 나타냄  
[p35-이미지 입력 공간]

`bin`: 히스토그램 그래프에서 가로축을 bin이라 한다.  
grayscale 영상의 경우에는 256개의 빈을 갖는 히스토그램을 구하는 것이 일반적 

경우에 따라서는 히스토그램의 빈 개수를 픽셀 값 범위보다 작게 설정할 수도 있음 (정규화 통해)

연산량 많아질 수 있다. 

히스토그램의 빈 개수가 줄어들면 히스토그램이 표현하는 영상의 픽셀 값 분포 모양이  
더 대략적인 형태로 바뀜  
빈 개수가 많으면 세밀한 픽셀 값 분포 표현이 가능 

calcHist() 함수로 히스토그램 알 수 있음  
다양한 형식의 히스토그램을 지원하기 때문에 calcHist() 함수의 사용법은 꽤 복잡한 편 

In [4]:
def calcGrayHist(img):
    channels = [0] # 채널의 경우에도 멀티채널인 경우가 있으므로 list 형태로 전달 
    histSize = [256]
    histRange = [0, 256]

    hist = cv2.calcHist([img], channels, None, histSize, histRange)
    return hist

In [3]:
calcGrayHist('data/lenna.bmp')

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'calcHist'
> Overload resolution failed:
>  - Can't parse 'images'. Sequence item with index 0 has a wrong type
>  - Can't parse 'images'. Sequence item with index 0 has a wrong type


calcGrayHist() 함수는 내부에서 OpenCV 함수 calcHist()를 이용하여  
그레이스케일 영상의 히스토그램을 표현하는 행렬 Hist를 구하여 반환 

이때 반환되는 hist는 256 * 1 크기의 행렬  
행 개수는 256이고, 열 개수는 1

막대그래프 형태로 나타내려면 직접 hist 행렬을 참조하여 막대그래프 영상을 참조해야 함 

## 히스토그램 평활화 

먼저 히스토그램을 구해야 함 

h(g)는 영상에서 그레이스케일 값이 g인 픽셀 개수  
히스토그램 평활화를 계산하기 위해서는 h(g)로부터 히스토그램 누적 함수 H(g)를 구해야 함(0일때 부터)

이 누적 함수를 픽셀 값 변환 함수로 사용 
h(g)의 값은, 누적 함수이므로, 보통 grayscale의 값의 범위보다 훨씬 크다.  
함수의 최댓값이 255가 되도록 정규화 

0부터 내가 찾고자 하는 픽셀까지의 누적합 * L_max / N  
L_max : 영상이 가질 수 있는 최대 밝기 -> grayscale에서는 255  
round 함수로 반올림 